# Obteniendo Dataset para TFM

In [ ]:
%pip install wbpy

In [ ]:
#@title liberias importadas
import requests
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 100)
import pandas_datareader as pdr
from datetime import datetime
import wbpy

# Crea una instancia de la API
api = wbpy.IndicatorAPI()

# ESPORTEARNINGS

In [ ]:
#@title Extraccion de datos

years = range(2000, 2023)
url_base = 'https://www.esportsearnings.com/history/'

df_country = pd.DataFrame()

for year in years:
    url = url_base + str(year) + '/countries'
    r = requests.get(url)
    datos = pd.read_html(r.text)
    datos_y = pd.DataFrame(datos[0][[1,2,3]])
    datos_y.columns = ['country', 'total_earnings', 'total_players']
    datos_y['year'] = year
    df_country = pd.concat([df_country, datos_y])  # concatena df_country y datos_y

df_country.sample(3)

,country,total_earnings,total_players,year
2,Sweden,"$444,887.73",134 Players,2009
4,China,"$192,627.92",104 Players,2004
25,Spain,"$356,999.66",308 Players,2015


In [ ]:
#@title Verificacion de paises
# Paises que aparecen x veces
top_country = (df_country
 .groupby(['country'], as_index=False)
 .agg(n=('country', 'count'))
 .sort_values('country')
 .query('n>=10')
 )['country'].unique()

df_topcountry = df_country.query('country in @top_country')

In [ ]:
#@title Limpiando datos
df_topcountry['year'] = pd.to_datetime(df_topcountry['year'], format='%Y')
df_topcountry = (df_topcountry
            .assign(
                total_earnings = df_topcountry['total_earnings'].replace('[\$,]', '', regex=True).astype(float),
                total_players = df_topcountry['total_players'].replace(' Players| Player', '', regex=True).replace(',', '', regex=True).astype(int))
            )
df_topcountry.sample(3)

,country,total_earnings,total_players,year
61,Ireland,1362.61,6,2011-01-01
10,Canada,5357827.47,769,2021-01-01
0,China,47371566.54,2156,2022-01-01


# Datos socioeconometricos

In [ ]:
#@title ISO codes

# Encontrnado country codes
r = requests.get('https://wits.worldbank.org/wits/wits/witshelp/content/codes/country_codes.htm')
datos = pd.read_html(r.text)
isos = pd.DataFrame(datos[0])
isos.columns = ['country', 'iso', 'code']
no_enlista = ['Puerto Rico', 'Kosovo', 'Gibraltar', 'Cayman Islands', 'Gaza Strip',
              'Somalia', 'Monaco', 'Korea, Dem. Rep.', 'San Marino',
              'British Virgin Islands', 'Liechtenstein', 'Virgin Islands (U.S.)']
isos = isos.query('country != @no_enlista')
isos.drop(isos.index[[0, 1]], inplace=True)

In [ ]:
#@title limpieza paises no info
# correccion de iso y quito paises sin informacion por temas politicos
faltantes = pd.DataFrame([
    {'country':'Egypt', 'iso': 'EGY', 'code':0},
    {'country':'Hong Kong', 'iso': 'HKG', 'code':0},
    {'country':'Iran, Islamic Republic of', 'iso': 'IRN', 'code':0},
    {'country':'Korea, Republic of', 'iso': 'KOR', 'code':0},
    # {'country':'Kosovo, Republic of', 'iso': 'KSV', 'code':0},
    {'country':'Kyrgyzstan', 'iso': 'KGZ', 'code':0},
    {'country':"Lao People's Democratic Republic", 'iso': 'LAO', 'code':0},
    {'country':"Moldova, Republic of", 'iso': 'MDA', 'code':0},
    # {'country':"Montenegro", 'iso': 'YUG', 'code':0},
    {'country':"North Macedonia", 'iso': 'MKD', 'code':0},
    # {'country':"Palestine, State of", 'iso': 'PSE', 'code':0},
    # {'country':"Serbia", 'iso': 'SRB', 'code':0},
    {'country':"Slovakia", 'iso': 'SK', 'code':0},
    # {'country':"Taiwan, Republic of China", 'iso': 'TWN', 'code':0},
    {'country':"Viet Nam", 'iso': 'VNM', 'code':0}
])

isos = isos._append(faltantes, ignore_index=True)


In [ ]:
#@title paises no encontrados
# paises_no_encontrados =
set(top_country) - set(isos.query('country in @top_country')['country'])

{'Puerto Rico', 'Serbia', 'Taiwan, Republic of China'}

In [ ]:
#@title paises finales
top_iso = isos.query('country in @top_country')['iso'].unique()
top_iso

In [ ]:
#@title funcion crea df
def crear_dataframe(indicador, nombre_columna):
    paises = top_iso.tolist()
    datos = api.get_dataset(indicador, paises, date="2000:2022")
    filas = []
    for pais, anos in datos.as_dict().items():
        for year, valor in anos.items():
            filas.append((pais, year, valor))
    df = pd.DataFrame(filas, columns=['iso', 'year', nombre_columna])
    return df

In [ ]:
#@title KPI WorldBank

df_pbicap = crear_dataframe('NY.GDP.PCAP.CD', 'pbicap') # pbi per capita
df_gdp_cap_g = crear_dataframe('NY.GDP.PCAP.KD.ZG', 'gdp_gr') # crecimiento per capita
df_inflacion = crear_dataframe('NY.GDP.DEFL.KD.ZG', 'inflacion') # inflacion
df_elect = crear_dataframe('EG.ELC.ACCS.ZS', 'elect_acc') # %pob accesos alectricidad
df_internet = crear_dataframe('IT.NET.USER.ZS', 'internet') # uso internet %pob
df_desempleo = crear_dataframe('SL.UEM.TOTL.ZS', 'desempleo') # desem % del EET
df_movil = crear_dataframe('IT.CEL.SETS.P2', 'movil') # suscripcion movil por cada 100
df_pob = crear_dataframe('SP.POP.TOTL', 'poblacion') # poblacion
df_pea = crear_dataframe('SP.POP.1564.TO', 'pea') # personas de 15 a 64
df_life_exp = crear_dataframe('SP.DYN.LE00.IN', 'life_exp') # life exp
df_net_mig = crear_dataframe('SM.POP.NETM', 'net_mig') # migracion neta

http://api.worldbank.org/v2/countries/ALB;ARG;ARM;AUS;AUT;AZE;BHR;BLR;BEL;BIH;BRA;BGR;CAN;CHL;CHN;COL;CRI;HRV;CUB;CZE;DNK;ECU;EST;FIN;FRA;DEU;GRC;HUN;ISL;IND;IDN;IRL;ISR;ITA;JPN;KAZ;KWT;LVA;LBN;LTU;MAC;MYS;MLT;MEX;MNG;MAR;NLD;NZL;NOR;PAK;PAN;PER;PHL;POL;PRT;ROM;RUS;SAU;SGP;SVN;ZAF;ESP;SWE;CHE;THA;TUR;UKR;ARE;GBR;USA;URY;UZB;VEN;HKG;IRN;KOR;MDA;MKD;SVK;VNM/indicators/NY.GDP.PCAP.CD?date=2000%3A2022&format=json&per_page=10000
http://api.worldbank.org/v2/countries/ALB;ARG;ARM;AUS;AUT;AZE;BHR;BLR;BEL;BIH;BRA;BGR;CAN;CHL;CHN;COL;CRI;HRV;CUB;CZE;DNK;ECU;EST;FIN;FRA;DEU;GRC;HUN;ISL;IND;IDN;IRL;ISR;ITA;JPN;KAZ;KWT;LVA;LBN;LTU;MAC;MYS;MLT;MEX;MNG;MAR;NLD;NZL;NOR;PAK;PAN;PER;PHL;POL;PRT;ROM;RUS;SAU;SGP;SVN;ZAF;ESP;SWE;CHE;THA;TUR;UKR;ARE;GBR;USA;URY;UZB;VEN;HKG;IRN;KOR;MDA;MKD;SVK;VNM/indicators/NY.GDP.PCAP.KD.ZG?date=2000%3A2022&format=json&per_page=10000
http://api.worldbank.org/v2/countries/ALB;ARG;ARM;AUS;AUT;AZE;BHR;BLR;BEL;BIH;BRA;BGR;CAN;CHL;CHN;COL;CRI;HRV;CUB;CZE;DNK;ECU;EST;FIN;FRA;DEU

In [ ]:
#@title ISO codes match
country_top = isos.query('country in @top_country')

df = (df_topcountry
      .merge(country_top, on='country', how='left')
      .dropna(subset=['iso']) # los nas que encontre antes
      )

# match de codigos ISO
iso_reemplazo = dict(zip(df['iso'].sort_values().unique(),
                          df_pbicap['iso'].unique()))
df['iso'] = df['iso'].map(iso_reemplazo)


isos_match=pd.read_csv("/content/drive/MyDrive/TFM/isocode.csv", sep=',')
isos_match.sample(3)

# Merge data with earnings

In [ ]:
#@title Merge dfs
indicadores = [df_pbicap, df_internet, df_desempleo, df_elect,
              df_movil, df_pob, df_pea, df_net_mig, df_life_exp,
              df_gdp_cap_g, df_inflacion]

# merge
for ind in indicadores:
    ind['year'] = pd.to_datetime(ind['year'], format='%Y')
    df = df.merge(ind, left_on=['iso', 'year'], right_on=['iso', 'year'], how='left')

df.shape

In [ ]:
#df.to_csv('df_17_21_noclean.csv', index=False)
df.to_csv('df_00_21_noclean.csv', index=False)